Задача 1. Выбор кредита от Tinkoff.ru
В credit_train.csv содержится 170746 строк с данными о клиентах сети магазинов электроники, в этих магазинах они подали заявки на кредит. Колонка open_account_flg содержит 1 если клиент выбрал Тинькофф и 0 в противном случае. В credit_test.csv содержится 91940 строк с данными, для каждой строки следует предсказать возьмет ли соответствующий ей человек кредит в Тинькофф.
Метрикой качества в задаче является AUC 

education - образование:

    SCH - начальное, среднее
    PGR -  второе высшее
    GRD - высшее
    UGR - неполное высшее
    ACD - ученая степень
    
    
job_position - работа

    SPC - неруководящий сотрудник, специалист
    DIR - руководитель организации
    HSK - домохозяйка
    INV - не работает, инвалидность
    WOI - работает на ИП
    WRK - неруководящий сотрудник, рабочий
    UMN -  руководитель подразделения
    NOR - не работает
    PNS - пенсионер
    BIS - собственный бизнес 
    INP - ИП
   
marital_status - семейное положение

    UNM - холост/не замужем
    DIV - разведен(а)
    MAR - женат/замужем
    WID - вдовец/вдова
    CIV - гражданский брак
    
gender - пол 

age - возраст

credit_sum - сумма кредита

credit_month - срок кредитования

tariff_id - номер предлагаемого тарифа

living_region - регион проживания

montly_income - заработная плата в месяц

credit_count - количество кредитов у клиента

overdue_credit_count - количество просроченных кредитов

score_shk - неизвестный индекс кредитоспособности
    

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation, metrics

import xgboost

/Users/Roman/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv("credit_train.csv", sep = ";", encoding='cp1251')
test = pd.read_csv("credit_test.csv", sep = ";", encoding='cp1251')

In [3]:
train.head(2)

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,1,M,48,MAR,UMN,"59998,00",10,1.6,"0,770249",GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,2,F,28,MAR,UMN,"10889,00",6,1.1,"0,248514",GRD,МОСКВА,43000.0,2.0,0.0,0


In [4]:
y = train['open_account_flg']

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170746 entries, 0 to 170745
Data columns (total 15 columns):
client_id               170746 non-null int64
gender                  170746 non-null object
age                     170746 non-null int64
marital_status          170746 non-null object
job_position            170746 non-null object
credit_sum              170746 non-null object
credit_month            170746 non-null int64
tariff_id               170746 non-null float64
score_shk               170746 non-null object
education               170746 non-null object
living_region           170554 non-null object
monthly_income          170745 non-null float64
credit_count            161516 non-null float64
overdue_credit_count    161516 non-null float64
open_account_flg        170746 non-null int64
dtypes: float64(4), int64(4), object(7)
memory usage: 19.5+ MB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91940 entries, 0 to 91939
Data columns (total 14 columns):
client_id               91940 non-null int64
gender                  91940 non-null object
age                     91940 non-null int64
marital_status          91940 non-null object
job_position            91940 non-null object
credit_sum              91940 non-null object
credit_month            91940 non-null int64
tariff_id               91940 non-null float64
score_shk               91940 non-null object
education               91940 non-null object
living_region           91824 non-null object
monthly_income          91940 non-null int64
credit_count            87237 non-null float64
overdue_credit_count    87237 non-null float64
dtypes: float64(3), int64(4), object(7)
memory usage: 9.8+ MB


In [7]:
train.gender = train.gender.map({"M":1, "F":0})
test.gender = test.gender.map({"M":1, "F":0})

In [8]:
train.rename(columns={"open_account_flg":"target"}, inplace=True)

I выделим группы людей по наличию высшего образования, работе и пенсии 

In [9]:
### по образованию
train["high_edu"] = ((train["education"] != "SCH") & (train["education"] != "UGR"))

### по работе
# 1. не работающие граждание
train["not_work"] = ((train.job_position == "HSK") | (train.job_position == "INV") 
                     | (train.job_position == "NOR") | (train.job_position == "PNS"))

# 2. обычные работники
train["usual_workers"] = ((train.job_position == "SPC") | (train.job_position == "WOI") 
                     | (train.job_position == "WRK") | (train.job_position == "ATP") 
                     | (train.job_position == "WRP"))

# 3. начальники
train["head_workers"] = ((train.job_position == "DIR") | (train.job_position == "UMN")
                         | (train.job_position == "BIS") | (train.job_position == "INP"))

# 4. имеют собственный бизнес
train["have_business"] = ((train.job_position == "BIS") | (train.job_position == "INP"))

### по браку
train["single"] = ((train.marital_status == "UNM") | (train.marital_status == "DIV")
                   | (train.marital_status == "WID"))

In [10]:
### по образованию
test["high_edu"] = ((test["education"] != "SCH") & (test["education"] != "UGR"))

### по работе
# 1. не работающие граждание
test["not_work"] = ((test.job_position == "HSK") | (test.job_position == "INV") 
                     | (test.job_position == "NOR") | (test.job_position == "PNS"))

# 2. обычные работники
test["usual_workers"] = ((test.job_position == "SPC") | (test.job_position == "WOI") 
                     | (test.job_position == "WRK") | (test.job_position == "ATP") 
                     | (test.job_position == "WRP"))

# 3. начальники
test["head_workers"] = ((test.job_position == "DIR") | (test.job_position == "UMN")
                         | (test.job_position == "BIS") | (test.job_position == "INP"))

# 4. имеют собственный бизнес
test["have_business"] = ((test.job_position == "BIS") | (test.job_position == "INP"))

### по браку
test["single"] = ((test.marital_status == "UNM") | (test.marital_status == "DIV")
                   | (test.marital_status == "WID"))

II кодировка категориальных признаков, кроме living region

In [11]:
train_col = train.columns.drop("target")

In [12]:
dummy_df = pd.concat([train[train_col], test], axis=0)

In [13]:
dummy_df = pd.get_dummies(dummy_df, columns=["education","job_position", "marital_status"], drop_first=True)

In [14]:
train = dummy_df[:train.shape[0]]
test = dummy_df[train.shape[0]:]

In [15]:
train.columns

Index([           u'client_id',               u'gender',
                        u'age',           u'credit_sum',
               u'credit_month',            u'tariff_id',
                  u'score_shk',        u'living_region',
             u'monthly_income',         u'credit_count',
       u'overdue_credit_count',             u'high_edu',
                   u'not_work',        u'usual_workers',
               u'head_workers',        u'have_business',
                     u'single',        u'education_GRD',
              u'education_PGR',        u'education_SCH',
              u'education_UGR',     u'job_position_BIS',
           u'job_position_BIU',     u'job_position_DIR',
           u'job_position_HSK',     u'job_position_INP',
           u'job_position_INV',     u'job_position_NOR',
           u'job_position_ONB',     u'job_position_PNA',
           u'job_position_PNI',     u'job_position_PNS',
           u'job_position_PNV',     u'job_position_SPC',
           u'job_position_UMN',

III кодировка региона через среднюю зарплату

In [16]:
def code_mean(data, cat_feature, real_feature):
    return (data[cat_feature].map(data.groupby(cat_feature)[real_feature].mean()))

In [17]:
train['monthly_income'] = train['monthly_income'].astype(float)
test['monthly_income'] = test['monthly_income'].astype(float)

train["city_mean_income"] = code_mean(train, 'living_region', 'monthly_income')
train.city_mean_income.fillna(value=train.city_mean_income.mean(), inplace=True)
train.drop("living_region", axis=1, inplace=True)

test["city_mean_income"] = code_mean(test, 'living_region', 'monthly_income')
test.city_mean_income.fillna(value=train.city_mean_income.mean(), inplace=True)
test.drop("living_region", axis=1, inplace=True)

/Users/Roman/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/Roman/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Roman/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

IV Генерация данных о зарплате и выплатам по кредиту

In [18]:
train['credit_sum'] = train['credit_sum'].apply(lambda x: x.replace(",","."))
test['credit_sum'] = test['credit_sum'].apply(lambda x: x.replace(",","."))

train['credit_sum'] = train['credit_sum'].astype(float)
test['credit_sum'] = test['credit_sum'].astype(float)

/Users/Roman/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/Roman/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Roman/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

credit_pay - ежемесячный расход на кредит

money_for_life - разность между зп и платой по кредиту

dif_city - разница между зп и средней зп по региону

In [19]:
train["credit_pay"] = train["credit_sum"] / train["credit_month"]
train["money_for_life"] = train["monthly_income"] - train["credit_pay"]
train["dif_city"] = train["monthly_income"] - train["city_mean_income"]

test["credit_pay"] = test["credit_sum"] / test["credit_month"]
test["money_for_life"] = test["monthly_income"] - test["credit_pay"]
test["dif_city"] = test["monthly_income"] - test["city_mean_income"]

/Users/Roman/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/Roman/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Roman/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

V анализ score_shk

In [20]:
train['score_shk'] = train['score_shk'].apply(lambda x: x.replace(",","."))
test['score_shk'] = test['score_shk'].apply(lambda x: x.replace(",","."))

train['score_shk'] = train['score_shk'].astype(float)
test['score_shk'] = test['score_shk'].astype(float)

/Users/Roman/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/Roman/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Roman/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [21]:
train['target'] = y

/Users/Roman/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [22]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170746 entries, 0 to 170745
Data columns (total 46 columns):
client_id               170746 non-null int64
gender                  170746 non-null int64
age                     170746 non-null int64
credit_sum              170746 non-null float64
credit_month            170746 non-null int64
tariff_id               170746 non-null float64
score_shk               170746 non-null float64
monthly_income          170745 non-null float64
credit_count            161516 non-null float64
overdue_credit_count    161516 non-null float64
high_edu                170746 non-null bool
not_work                170746 non-null bool
usual_workers           170746 non-null bool
head_workers            170746 non-null bool
have_business           170746 non-null bool
single                  170746 non-null bool
education_GRD           170746 non-null uint8
education_PGR           170746 non-null uint8
education_SCH           170746 non-null uint8
education

In [23]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91940 entries, 0 to 91939
Data columns (total 45 columns):
client_id               91940 non-null int64
gender                  91940 non-null int64
age                     91940 non-null int64
credit_sum              91940 non-null float64
credit_month            91940 non-null int64
tariff_id               91940 non-null float64
score_shk               91940 non-null float64
monthly_income          91940 non-null float64
credit_count            87237 non-null float64
overdue_credit_count    87237 non-null float64
high_edu                91940 non-null bool
not_work                91940 non-null bool
usual_workers           91940 non-null bool
head_workers            91940 non-null bool
have_business           91940 non-null bool
single                  91940 non-null bool
education_GRD           91940 non-null uint8
education_PGR           91940 non-null uint8
education_SCH           91940 non-null uint8
education_UGR           91940 

VI работа с пропусками

In [24]:
train.dropna(subset=["monthly_income"], inplace=True)

miss_df = train[train.credit_count.isnull()]
miss_train_df = train[train.credit_count.notnull()]

target_credit_count = miss_train_df.credit_count
target_overdue_credit_count = miss_train_df.overdue_credit_count

miss_train_df.drop(labels=['credit_count', 'overdue_credit_count', 'client_id'], axis=1, inplace=True)
miss_train, miss_test, y_train_credit_count, y_test_credit_count = train_test_split(miss_train_df, target_credit_count,\
                                                                                   test_size = 0.3)


/Users/Roman/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/Roman/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [25]:
miss_train = miss_train.values.astype(float)
miss_test = miss_test.values.astype(float)

In [26]:
XGB = xgboost.XGBRegressor(max_depth=8, learning_rate=0.05, n_estimators=600)
print "fitting...\n"
XGB.fit(miss_train, y_train_credit_count)
pred = XGB.predict(miss_test)
print metrics.mean_squared_error(y_test_credit_count, pred)
print list(y_test_credit_count[:10]), "\n"
print pred[:10]

fitting...

2.89181266388
[2.0, 0.0, 3.0, 3.0, 0.0, 2.0, 1.0, 3.0, 6.0, 2.0] 

[ 2.67046452  1.87687147  2.23679447  2.08420801  1.70754826  2.02170467
  2.31324267  1.97594655  1.97650683  2.05408525]


In [27]:
mean = [y_train_credit_count.mean() for x in range(len(y_test_credit_count))]
print metrics.mean_squared_error(y_test_credit_count, mean), "\n"
print mean[:10], "\n"
print list(y_test_credit_count[:10])

3.15644471117 

[2.1038563594551567, 2.1038563594551567, 2.1038563594551567, 2.1038563594551567, 2.1038563594551567, 2.1038563594551567, 2.1038563594551567, 2.1038563594551567, 2.1038563594551567, 2.1038563594551567] 

[2.0, 0.0, 3.0, 3.0, 0.0, 2.0, 1.0, 3.0, 6.0, 2.0]


In [28]:
tmp = miss_df.drop(labels = ['credit_count', 'overdue_credit_count', 'client_id'], axis=1)
tmp = tmp.values.astype(float)
val=XGB.predict(tmp)

In [ ]:
k=0
for i, j in zip(train.credit_count, enumerate(train.credit_count)):
    if np.isnan(i):
        train.credit_count[j[0]] = val[k]
        k+=1

/Users/Roman/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Roman/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
train.info()

In [ ]:
train.to_csv("train_without_oversampling.csv", columns=train.columns, index=False)
test.to_csv("test.csv", columns=test.columns, index=False)